# Experiment : Network growth

## Init the Ray client

In [2]:
# Script for generate  experiments
# Imports
from clases.red_rddas_model import RedRddasModel
import ray

import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Ray Configurations
runtime_env = {"working_dir": "/home/carlos/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False)

# ray.init(log_to_driver=False, num_cpus=12)

2022-06-28 23:49:02,242	INFO client_builder.py:224 -- Passing the following kwargs to ray.init() on the server: log_to_driver
2022-06-28 23:49:02,778	INFO packaging.py:414 -- Creating a file package for local directory '/home/carlos/Documents/RESEARCH/SynEstRDDA'.
2022-06-28 23:49:02,902	INFO packaging.py:258 -- Pushing file package 'gcs://_ray_pkg_18fb8554be254ed3.zip' (17.32MiB) to Ray cluster...
2022-06-28 23:49:27,384	INFO packaging.py:267 -- Successfully pushed file package 'gcs://_ray_pkg_18fb8554be254ed3.zip'.


ClientContext(dashboard_url='', python_version='3.8.10', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', protocol_version='2022-03-16', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7fe50ad83fa0>)

In [3]:
# Experiment for RDD variation
n_experiments = 10
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 10

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"
    # this name has to be unique
    unique_path = "21_01_2022"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        # path = "files/" + unique_path + "_" + str(n_of_rdds) + "_" + str(n_of_variables_rdda) + "_" + str(n_of_signals_rdda) \
        #        + "_" + str(n_exit_variables) + "_" + str(n_clauses_function)

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        oRedRddasModel.generate_rddas(type_network=type_network)

        # # Save the Network of RDDAs in a Pickle file
        # RedRddasModel.save_file_pickle(oRedRddasModel, path)
        # path += ".pickle"
        #
        # print("=======================================================")
        # print("The Network of RDDAs is saved in: ", path)

        # Show the Network of RDDAs
        # oRedRddasModel.show()

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
        # # oRedRddasModel = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(oRedRddasModel.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
print("END EXPERIMENT")

Experiment: 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 2
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 3
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 4
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 6
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 7
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 8
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Experiment: 2
Number of Network: 1
-------------------------------
generating the 

In [4]:
# Experiment for RDD variation
n_experiments = 10
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 10

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"
    # this name has to be unique
    unique_path = "21_01_2022"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        path = "files/" + unique_path + "_" + str(n_of_rdds) + "_" + str(n_of_variables_rdda) + "_" + str(n_of_signals_rdda) \
               + "_" + str(n_exit_variables) + "_" + str(n_clauses_function)

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        oRedRddasModel.generate_rddas(type_network=type_network)

        # # Save the Network of RDDAs in a Pickle file
        # RedRddasModel.save_file_pickle(oRedRddasModel, path)
        # path += ".pickle"
        #
        # print("=======================================================")
        # print("The Network of RDDAs is saved in: ", path)

        # Show the Network of RDDAs
        # oRedRddasModel.show()

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
        # # oRedRddasModel = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(oRedRddasModel.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
print("END EXPERIMENT")

Experiment: 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 2
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 3
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 4
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 6
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 7
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 8
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Experiment: 2
Number of Network: 1
-------------------------------
generating the 

## Save the experiment data in csv, using pandas Dataframe

In [5]:
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

,n_network,n_rdds,n_rdda_attractors,t_find_attractors_method,n_pair_attractors,t_comp_paris_method,n_attractor_fields,t_optimized_method
n_sample,,,,,,,,
1,1,3,30,0.716689,164,0.082431,6,0.084636
1,2,4,31,0.877630,205,0.088820,16,0.121915
1,3,5,32,0.938117,133,0.088663,48,0.058626
1,4,6,42,1.074633,205,0.093972,174,0.324162
1,5,7,49,1.305310,241,0.106748,0,0.075559
...,...,...,...,...,...,...,...,...
10,4,6,61,1.345215,448,0.149889,0,0.085104
10,5,7,92,1.974560,562,0.175161,0,0.181025
10,6,8,86,1.944312,671,0.213908,0,0.154276


In [6]:
path = "exp1_network_growth_data.csv"
pf_res.to_csv(path)
print("Experiment saved in:", path)

Experiment saved in: exp1_network_growth_data.csv
